In [1]:
from bs4 import BeautifulSoup as bs
import time
import requests
import pandas as pd
import numpy as np

In [44]:
test = requests.get('https://dumps.wikimedia.org/wikidatawiki/20201001/wikidatawiki-20201001-pages-articles-multistream6.xml-p5969005p6052571.bz2')

In [82]:
test.text[:10]

'BZh91AY&SY'

In [2]:
content = []
# Read the XML file
with open('enwiki-20201001-pages-meta-history13.xml-p11424666p11424865', encoding='utf8') as file:
    # Read each line in the file, readlines() returns a list of lines
    content = file.readlines()
    # Combine the lines in the list into a string
    content_string = "".join(content)
    bs_content = bs(content_string, 'xml')

In [3]:
pages = bs_content.findAll('page')

In [4]:
lst = []
for page in pages:
    title = page.title.text
    for revision in page.findAll('revision'):
        try:
            timestamp = revision.timestamp.text
        except:
            timestamp = np.NaN
        try:
            username = revision.username.text
        except:
            username = np.NaN
        try:    
            comment = revision.comment.text
        except:
            comment = np.NaN
        try:
            text_byte = revision.format.next_sibling.next_sibling.text
        except:
            text_byte = np.NaN
        lst.append([title, timestamp, username, comment, text_byte])

In [5]:
df = pd.DataFrame(lst, columns=['title', 'timestamp', 'username', 'comment', 'text byte'])
df['timestamp'] = pd.to_datetime(df['timestamp'])

In [10]:
df

,title,timestamp,username,comment,text byte
0,Altoona and Wopsononock Railroad,2007-05-26 02:23:31+00:00,Choess,redirect,#REDIRECT [[Altoona and Beech Creek Railroad]]
1,Altoona and Wopsononock Railroad,2008-12-12 06:58:49+00:00,NE2,NaN,#REDIRECT [[Altoona and Beech Creek Railroad]]...
2,Altoona and Wopsononock Railroad,2009-03-06 01:49:43+00:00,Choess,cat,#REDIRECT [[Altoona and Beech Creek Railroad#A...
3,Altoona and Wopsononock Railroad,2009-06-14 02:28:14+00:00,NE2,Adding DEFAULTSORT per [[WP:TWP/MOS]] using [[...,#REDIRECT [[Altoona and Beech Creek Railroad#A...
4,Altoona and Wopsononock Railroad,2010-10-26 05:08:53+00:00,Choess,Adding category [[:Category:Railway companies ...,#REDIRECT [[Altoona and Beech Creek Railroad#A...
...,...,...,...,...,...
1438,User talk:24.205.142.100,2007-06-18 20:43:02+00:00,Nixeagle,I'll unblock,<nowiki>{{unblock|I AM NOT AN OPEN PROXY; Eagl...
1439,User talk:24.205.142.100,2007-06-18 20:46:30+00:00,Nixeagle,note,<nowiki>{{unblock|I AM NOT AN OPEN PROXY; Eagl...
1440,User talk:24.205.142.100,2017-05-13 16:33:34+00:00,BD2412bot,/* top */[[Wikipedia talk:Criteria for speedy ...,{{blocked proxy}}\n{{OW}}
1441,User talk:24.211.109.148,2007-05-26 02:31:16+00:00,Nixeagle,Open proxy,"{{openproxy}}\n<div align=""right"">--—— '''[[us..."


In [213]:
most_revs = df.groupby('title').username.apply(lambda x: x.value_counts(normalize=True).max())
((most_revs >= 0.4) & (most_revs <= 0.5)).mean()

0.704225352112676

In [202]:
df.groupby('username').title.nunique().sort_values(ascending=False)

username
BD2412bot                                              95
MichaelBillington                                      93
Yobot                                                   9
Choess                                                  7
Addbot                                                  6
Skier Dude                                              5
Tom.Reding                                              4
SmackBot                                                4
NE2                                                     4
Rjwilmsi                                                4
Waacstats                                               3
AvicBot                                                 3
Good Olfactory                                          3
Vegaswikian                                             3
AnomieBOT                                               3
InternetArchiveBot                                      3
Bearcat                                                 3
Monkb

In [214]:
df.timestamp.dt.month.value_counts(normalize=True)

5     0.306999
6     0.106722
3     0.086625
7     0.083160
11    0.075537
9     0.067221
8     0.064449
1     0.054747
10    0.042273
2     0.040194
4     0.038115
12    0.033957
Name: timestamp, dtype: float64